In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import tensorflow as tf
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['ner_dataset.csv', 'ner.csv']


In [2]:
dframe = pd.read_csv("../input/ner.csv", encoding = "ISO-8859-1", error_bad_lines=False)

b'Skipping line 281837: expected 25 fields, saw 34\n'


In [3]:
dframe.head()

,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,pos,prev-iob,prev-lemma,prev-pos,prev-prev-iob,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,NNS,__START1__,__start1__,__START1__,__START2__,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,IN,O,thousand,NNS,__START1__,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,NNS,O,of,IN,O,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,VBP,O,demonstr,NNS,O,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,VBN,O,have,VBP,O,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [4]:
df1=dframe[['word','tag']]

In [5]:
data=[]
tag=[]
x=[]
y=[]
for k,v in zip(df1.word,df1.tag):
    
    if k==".":
        data.append(x)
        tag.append(y)
        x=[]
        y=[]
    else:
        x.append(k)
        y.append(v)

In [6]:
len(data)

47895

In [7]:
data=data[:20000]
tag=tag[:20000]

In [8]:
maxlen=max(map(len,data))

In [9]:
maxlen

70

In [10]:
from collections import Counter,defaultdict


In [11]:
datadict=defaultdict(lambda:0)

In [12]:
uniq_data_words=set()
for x in data:
    uniq_data_words.update(set(x))

In [13]:
special_tokens = ['<UNK>', '<PAD>']

In [14]:
words=special_tokens+list(x for x in uniq_data_words if x not in special_tokens)

In [15]:
uniq_tags=set()
for x in tag:
    uniq_tags.update(set(x))

In [16]:
unique_tags=['<PAD>']+list(uniq_tags)

In [17]:
tok_2_indx=dict([(v,k) for k,v in enumerate(words)])

In [18]:
ind_2_token=dict([(v,k) for k,v in tok_2_indx.items()])

In [19]:
tag_2_indx=dict([(k,v) for v,k in enumerate(unique_tags)])

In [20]:
indx_2_tag=dict([(k,v) for v,k in tag_2_indx.items()])

In [21]:
def batch_generator(data,tag,batchsize,shuffle=True,allow_smaller_lastbatch=True):
    N=len(data)
    if shuffle==True:
        order=np.random.permutation(N)
    else:
        order=np.arange(N)
        
    n_batches=N//batchsize
    if allow_smaller_lastbatch and (N%batchsize):
        n_batches+=1
        
    for k in range(n_batches):
        batchstart=k*batchsize
        batchend=min(batchstart+batchsize,N)
        
        currentbatchsize=batchend-batchstart
        
        x_list=[]
        y_list=[]
        
        max_len_token=0
        
        for idx in order[batchstart:batchend]:
            x_list.append([tok_2_indx[i] for i in data[idx]])
            y_list.append([tag_2_indx[i] for i in tag[idx]])
            
            max_len_token=max(max_len_token,len(data[idx]))
        
        x=np.ones([currentbatchsize,max_len_token],dtype=np.int32)*tok_2_indx['<PAD>']
        y=np.ones([currentbatchsize,max_len_token],dtype=np.int32)*tag_2_indx['<PAD>']
        
        lengths=np.zeros(currentbatchsize,dtype=np.int32)
        
        for n in range(currentbatchsize):
            len1=len(x_list[n])
            x[n,:len1]=x_list[n]
            y[n,:len1]=y_list[n]
            
            lengths[n]=len1
        
        yield x,y,lengths
        
        

In [22]:
x=tf.placeholder(dtype=tf.int32,shape=[None,None])
y=tf.placeholder(dtype=tf.int32,shape=[None,None])
lengths=tf.placeholder(dtype=tf.int32,shape=[None])
drpout=tf.placeholder(dtype=tf.float32,shape=[])
lr=tf.placeholder(dtype=tf.float32,shape=[])

In [23]:
enc_var=tf.Variable(tf.random_normal(shape=[len(tok_2_indx),32]),dtype=tf.float32)
encod_emb=tf.nn.embedding_lookup(enc_var,x)


Instructions for updating:
Colocations handled automatically by placer.


In [24]:
def createcell():
    cell=tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(256),input_keep_prob=drpout,output_keep_prob=drpout)
    return cell
    

In [25]:
nlayers=2

In [26]:
cellfw=[createcell() for i in range(nlayers)]
cellbw=[createcell() for i in range(nlayers)]

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


In [27]:
outputs,outputlast_fw,outputlast_bw=tf.contrib.rnn.stack_bidirectional_dynamic_rnn(cellfw,cellbw,encod_emb,sequence_length=lengths,dtype=tf.float32)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [28]:
outputs

<tf.Tensor 'stack_bidirectional_rnn/cell_1/concat:0' shape=(?, ?, 512) dtype=float32>

In [29]:
logits=tf.layers.dense(outputs,len(tag_2_indx),activation=None)

Instructions for updating:
Use keras.layers.dense instead.


In [30]:
pred_softmax=tf.nn.softmax(logits,axis=-1)
pred=tf.argmax(pred_softmax,axis=-1)

In [31]:
ytruth=tf.one_hot(y,len(tag_2_indx))
loss=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=ytruth)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [32]:
mask=tf.cast(tf.not_equal(x,tok_2_indx['<PAD>']),tf.float32)

In [33]:
loss1=tf.reduce_mean(tf.multiply(loss,mask))

In [34]:
optimizer=tf.train.AdamOptimizer(learning_rate=lr)
grads_vars=optimizer.compute_gradients(loss1)

In [35]:
clip_norm=tf.cast(1.0,tf.float32)
grads_vars1=[(tf.clip_by_norm(grad,1.0),var) for grad,var in grads_vars]

In [36]:
train_op=optimizer.apply_gradients(grads_vars1)

In [37]:
batchsize=32
epochs=5
lr1=0.006
lr_decay=2**(0.5)
drp=0.8

In [38]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [39]:
from tqdm import tqdm

In [40]:
for epoch in tqdm(range(epochs)):
    count=0
    for xbatch,ybatch,lengths1 in batch_generator(data,tag,batchsize):
        feeddict={x:xbatch,y:ybatch,lengths:lengths1,drpout:drp,lr:lr1}
        _,tloss=sess.run([train_op,loss1],feed_dict=feeddict)
        if (count%50)==0:
            print("count:-",count ," training loss:-",tloss)
        count+=1
    lr1=lr1/(lr_decay)

  0%|          | 0/5 [00:00<?, ?it/s]

count:- 0  training loss:- 1.5411783
count:- 50  training loss:- 0.45688027
count:- 100  training loss:- 0.29421812
count:- 150  training loss:- 0.22619155
count:- 200  training loss:- 0.19595589
count:- 250  training loss:- 0.1351361
count:- 300  training loss:- 0.12707186
count:- 350  training loss:- 0.07325581
count:- 400  training loss:- 0.12885165
count:- 450  training loss:- 0.10154069
count:- 500  training loss:- 0.11342144
count:- 550  training loss:- 0.14052346
count:- 600  training loss:- 0.11057894


 20%|██        | 1/5 [02:33<10:15, 153.96s/it]

count:- 0  training loss:- 0.052858345
count:- 50  training loss:- 0.05651643
count:- 100  training loss:- 0.07435864
count:- 150  training loss:- 0.06973711
count:- 200  training loss:- 0.06430338
count:- 250  training loss:- 0.08007337
count:- 300  training loss:- 0.13355084
count:- 350  training loss:- 0.10281715
count:- 400  training loss:- 0.095142655
count:- 450  training loss:- 0.09011428
count:- 500  training loss:- 0.0996816
count:- 550  training loss:- 0.07238927
count:- 600  training loss:- 0.06583733


 40%|████      | 2/5 [05:05<07:39, 153.25s/it]

...

 60%|██████    | 3/5 [07:37<05:05, 152.85s/it]

count:- 0  training loss:- 0.029217428
count:- 50  training loss:- 0.04527338
count:- 100  training loss:- 0.052889768
count:- 150  training loss:- 0.03936662
count:- 200  training loss:- 0.053506248
count:- 250  training loss:- 0.039592676
count:- 300  training loss:- 0.051703643
count:- 350  training loss:- 0.039592955
count:- 400  training loss:- 0.04128531
count:- 450  training loss:- 0.059131276
count:- 500  training loss:- 0.040608663
count:- 550  training loss:- 0.053700134
count:- 600  training loss:- 0.055432267


 80%|████████  | 4/5 [10:07<02:32, 152.07s/it]

count:- 0  training loss:- 0.01841508
count:- 50  training loss:- 0.042483415
count:- 100  training loss:- 0.026440684
count:- 150  training loss:- 0.033613488
count:- 200  training loss:- 0.030196382
count:- 250  training loss:- 0.018558629
count:- 300  training loss:- 0.018651238
count:- 350  training loss:- 0.016063515
count:- 400  training loss:- 0.028273921
count:- 450  training loss:- 0.010998628
count:- 500  training loss:- 0.03356668
count:- 550  training loss:- 0.021550514
count:- 600  training loss:- 0.024969628


100%|██████████| 5/5 [12:39<00:00, 151.95s/it]


In [41]:
xtrain=[[tok_2_indx[i] for i in sent] for sent in data]

In [42]:
ytrain=[[tag_2_indx[i] for i in tagsent] for tagsent in tag]

In [43]:
len1=[len(sent) for sent in data]

In [44]:
maxlen=max(len1)

In [45]:
padarray=np.ones([20000,maxlen])*tok_2_indx['<PAD>']

In [46]:
paddtokarray=np.ones([20000,maxlen])*tag_2_indx['<PAD>']

In [47]:

for i in range(20000):
    padarray[i,:len(xtrain[i])]=xtrain[i]
    paddtokarray[i,:len(ytrain[i])]=ytrain[i]

In [48]:
feeddict={x:padarray,y:paddtokarray,lengths:len1,drpout:1}

In [49]:
prediction=sess.run([pred],feed_dict=feeddict)

In [50]:
ind2tag=dict([(v,k) for k,v in tag_2_indx.items()])

In [51]:
from sklearn.metrics import f1_score

In [52]:
yactual=[]
ypred=[]
for i in range(20000):
    yactual+=tag[i]
    ypred+=[indx_2_tag[j] for j in prediction[0][i][:len1[i]]]

In [53]:
len(yactual)

418914

In [54]:
f1score=f1_score(ypred,yactual,average='macro')

In [55]:
f1score

0.9171354062357738